In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

# Data Setup & Functions

In [3]:
from mll_calc.mll_pred import ratios, format_XY, convert_g_to_mgUi

In [4]:
def logpdf_calc(row, test_sample, unc):
    y_sim = row[test_sample>0].values.tolist()
    std = row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [5]:
def loop_sfco(XY, test, pred, unc, lbls, nonlbls):
    xy_cols = XY.columns.tolist()
    for col in nonlbls: xy_cols.remove(col)
    test = test[xy_cols]
    
    logpdf_df = pd.DataFrame()
    for test_idx, row in test.iterrows():
        test_sample = row.drop(lbls)
        test_answer = row[lbls]
        pred_row = pred.loc[pred['sim_idx'] == test_idx]
        pred_idx = pred_row['pred_idx'].values[0]
        train_row = XY.loc[pred_idx].drop(lbls+nonlbls)
        
        logpdf = logpdf_calc(train_row, test_sample, unc)
        mll = pred_row['MaxLogLL'].values[0]
        
        if round(mll, 4) != round(np.sum(logpdf), 4):
            print('mismatch')
            break
        
        logpdf = pd.Series(logpdf, index=train_row[test_sample>0].index, name=test_idx)
        if logpdf_df.empty:
            logpdf_df = pd.DataFrame(columns = test_sample.index.to_list())
        logpdf_df = logpdf_df.append(logpdf)
    return logpdf_df

### Train and Test DBs

In [6]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
nonlbls = ['AvgPowerDensity', 'ModDensity', 'UiWeight']
ratio_list = ['cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
              'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
              'pu241/pu239', 'pu242/pu239']

train_pkl = '~/sims_n_results/simupdates_aug2020/not-scaled_nuc15.pkl'
XY = format_XY(train_pkl)
#XY = convert_g_to_mgUi(XY, lbls+nonlbls)

sfco_pkl = '~/sfcompo/format_clean/sfcompo_nuc15.pkl'
sfco = pd.read_pickle(sfco_pkl)

XY = ratios(XY, ratio_list, lbls+nonlbls)
sfco = ratios(sfco, ratio_list, lbls)

### LL Calc Results

In [7]:
sfcompo_results = '~/sims_n_results/simupdates_aug2020/sfco10ratio/'

uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
dfs = {}
for i, unc_job in enumerate(job_dirs):
    dfs['unc' + str(i)] = pd.read_csv(sfcompo_results + unc_job + '/' + unc_job + '.csv')

for dfname in list(dfs.keys()):
    df = dfs[dfname]
    df['Relative_Burnup_Error'] = df['Burnup_Error'] / df['Burnup']
    df['Relative_Enrichment_Error'] = df['Enrichment_Error'] / df['Enrichment']

# For ease of only viewing 5% case for now
preds = dfs['unc0']

### LogPDF (wrt nuclide) results

In [9]:
unc = 0.05
logpdf_df = loop_sfco(XY, sfco, preds, unc, lbls, nonlbls)

# Explore Errors

### Burnup

In [10]:
preds['Relative_Burnup_Error'].describe()
#nuc15 results for reference:
#count    505.000000
#mean       0.187539
#std        0.155398
#min        0.000408
#25%        0.058485
#50%        0.153846
#75%        0.298556
#max        1.503097

count    505.000000
mean       0.332190
std        0.214186
min        0.000047
25%        0.168213
50%        0.335406
75%        0.463196
max        2.496839
Name: Relative_Burnup_Error, dtype: float64

In [16]:
to_print = ['sim_idx', 'pred_idx', 'pred_CoolingTime', 'AvgPowerDensity', 
            'ReactorType', 'pred_ReactorType', 'ReactorType_Score', 
            'Enrichment', 'pred_Enrichment', 'Relative_Enrichment_Error', 
            'Burnup', 'pred_Burnup', 'Relative_Burnup_Error',
            'OrigenReactor', 'pred_OrigenReactor', 'MaxLogLL']
preds.loc[preds['Relative_Burnup_Error'] > 0.7, to_print]

,sim_idx,pred_idx,pred_CoolingTime,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
24,FDN-1|2F1ZN2|C3|UB,7261,0.504127,20.0,bwr,bwr,True,4.900,0.50,0.897959,47490.0,13478.85,0.716175,Fukushima-Daini-1_BWR,ge7x7-0,10.082432
26,FDN-1|2F1ZN3|C3|UT,190443,8.087540,25.0,bwr,bwr,True,4.900,0.50,0.897959,59050.0,15572.83,0.736277,Fukushima-Daini-1_BWR,svea64-1,9.546621
54,GAR-1|SA-13|E6|11,175036,1213.711645,25.0,bwr,bwr,True,2.410,5.11,1.120332,4200.0,7606.12,0.810981,Garigliano-1_BWR,atrium10x10-9,18.950928
55,GAR-1|SA-13|E6|13,176826,242.633977,10.0,bwr,bwr,True,2.410,5.11,1.120332,5580.0,9700.27,0.738400,Garigliano-1_BWR,atrium10x10-9,17.525297
132,JPD-1|A-20|A3|KC-1290-415,312493,859.467417,41.0,bwr,pwr,False,2.600,0.51,0.803846,6223.0,1808.72,0.709349,Japan Power Demonstration Reactor-1_BWR,bw15x15,22.517075
134,JPD-1|A-20|A6|KC-1334-293,265873,1066.602482,35.0,bwr,pwr,False,2.600,0.50,0.807692,6593.0,1887.70,0.713681,Japan Power Demonstration Reactor-1_BWR,w17x17,23.925281
236,NPD-1|1022|A|1,154737,6902.526051,25.0,phwr,bwr,False,0.711,3.28,3.613221,791.0,2766.00,2.496839,Nuclear Power Demonstration Reactor-1_CANDU,atrium10x10-9,22.265036
237,NPD-1|1022|B|1,144597,6902.526051,25.0,phwr,bwr,False,0.711,1.87,1.630098,854.0,2132.20,1.496721,Nuclear Power Demonstration Reactor-1_CANDU,atrium10x10-9,17.098496
238,NPD-1|1022|C|1,149629,4664.390414,25.0,phwr,bwr,False,0.711,1.87,1.630098,1059.0,2132.20,1.013409,Nuclear Power Demonstration Reactor-1_CANDU,atrium10x10-9,22.360248
241,NPD-1|1129|C|1,20325,4343.164051,10.0,phwr,bwr,False,0.711,2.03,1.855134,1622.0,2970.68,0.831492,Nuclear Power Demonstration Reactor-1_CANDU,ge7x7-0,19.963821


### Enrichment

In [17]:
preds['Relative_Enrichment_Error'].describe()
#nuc15 results for reference:
#count    505.000000
#mean       0.369961
#std        0.289978
#min        0.000000
#25%        0.119444
#50%        0.288235
#75%        0.611765
#max        2.853727

count    505.000000
mean       0.445431
std        0.309600
min        0.000000
25%        0.224944
50%        0.442308
75%        0.652778
max        3.613221
Name: Relative_Enrichment_Error, dtype: float64

In [18]:
preds.loc[preds['Relative_Enrichment_Error'] > 1.0, to_print]

,sim_idx,pred_idx,pred_CoolingTime,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
54,GAR-1|SA-13|E6|11,175036,1213.711645,25.0,bwr,bwr,True,2.410,5.11,1.120332,4200.0,7606.12,0.810981,Garigliano-1_BWR,atrium10x10-9,18.950928
55,GAR-1|SA-13|E6|13,176826,242.633977,10.0,bwr,bwr,True,2.410,5.11,1.120332,5580.0,9700.27,0.738400,Garigliano-1_BWR,atrium10x10-9,17.525297
236,NPD-1|1022|A|1,154737,6902.526051,25.0,phwr,bwr,False,0.711,3.28,3.613221,791.0,2766.00,2.496839,Nuclear Power Demonstration Reactor-1_CANDU,atrium10x10-9,22.265036
237,NPD-1|1022|B|1,144597,6902.526051,25.0,phwr,bwr,False,0.711,1.87,1.630098,854.0,2132.20,1.496721,Nuclear Power Demonstration Reactor-1_CANDU,atrium10x10-9,17.098496
238,NPD-1|1022|C|1,149629,4664.390414,25.0,phwr,bwr,False,0.711,1.87,1.630098,1059.0,2132.20,1.013409,Nuclear Power Demonstration Reactor-1_CANDU,atrium10x10-9,22.360248
239,NPD-1|1129|A|1,270941,4020.888647,41.0,phwr,pwr,False,0.711,1.46,1.053446,1223.0,1887.70,0.543500,Nuclear Power Demonstration Reactor-1_CANDU,w17x17,21.851028
240,NPD-1|1129|B|1,292361,3779.651631,35.0,phwr,pwr,False,0.711,1.48,1.081575,1306.0,1967.36,0.506401,Nuclear Power Demonstration Reactor-1_CANDU,s18x18,21.585903
241,NPD-1|1129|C|1,20325,4343.164051,10.0,phwr,bwr,False,0.711,2.03,1.855134,1622.0,2970.68,0.831492,Nuclear Power Demonstration Reactor-1_CANDU,ge7x7-0,19.963821


# Max Log LL

In [19]:
print(dfs['unc0']['MaxLogLL'].describe())
print(dfs['unc3']['MaxLogLL'].describe())

count     505.000000
mean       -8.808792
std       283.882673
min     -6198.784474
25%         8.204578
50%        11.069605
75%        13.624339
max        31.310547
Name: MaxLogLL, dtype: float64
count    505.000000
mean       7.159359
std       18.627123
min     -384.757899
25%        6.113215
50%        7.719232
75%       10.116667
max       26.559136
Name: MaxLogLL, dtype: float64


# Nuclide Contributions to LL

In [20]:
logpdf_df.sample(10)

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
NOV-4|213|25|10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.860600,3.451088,4.274779
YNK-1|E5|C-A6|G-101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.746298,4.013430,6.622234
OBR-1|BE170|1|94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.946152,3.617876,4.817594
NOV-4|13626135|69|57,2.310579,8.163345,1.202899,NaN,-2.689287,-4.393090,4.506676,1.476846,4.062467,4.358618
TMI-1|NJ05YU|H6|A2,NaN,NaN,NaN,NaN,-2.641385,-1.725747,NaN,0.232290,3.436796,1.154885
TMI-1|NJ05YU|H6|B3J,NaN,NaN,NaN,NaN,-3.119932,-1.746992,NaN,0.682441,3.390195,3.437023
TSU-1|JAB-74|A1|MC5-MS3-Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.321256,3.412321,3.648354
DOD-1|Y013|UO2-b2|DU1,0.145218,4.622862,3.159701,NaN,-9.906189,-8.973441,3.402742,1.563592,3.132733,3.603634
YNK-1|E5|C-A6|G-102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.794748,4.342253,6.763073
JPD-1|A-20|E2|KA-0343+415,NaN,6.983328,NaN,NaN,NaN,NaN,NaN,4.121250,5.738417,7.932675


In [22]:
for rat in ratio_list:
    desc = logpdf_df[rat].describe()
    print(desc[['count', 'mean', 'std', 'min', 'max']])

count      43.000000
mean     -157.596231
std       886.878615
min     -5739.045019
max         2.382559
Name: cs137/cs133, dtype: float64
count    113.000000
mean       2.738086
std       20.884096
min     -151.350286
max        9.737968
Name: cs134/cs137, dtype: float64
count     40.000000
mean      -6.110923
std       30.749540
min     -169.155606
max        3.369438
Name: cs135/cs137, dtype: float64
count    0.0
mean     NaN
std      NaN
min      NaN
max      NaN
Name: ba136/ba138, dtype: float64
count    97.000000
mean     -5.631247
std       9.142124
min     -85.351765
max      -1.920391
Name: sm150/sm149, dtype: float64
count    97.000000
mean     -4.504230
std       6.703995
min     -52.852701
max      -1.033761
Name: sm152/sm149, dtype: float64
count    43.000000
mean     -1.560306
std      10.942918
min     -36.201444
max       4.885815
Name: eu154/eu153, dtype: float64
count    505.000000
mean       1.399128
std        5.153639
min      -37.683053
max        4.786167
Name: p

# Look at ratio values for a single prediction

In [35]:
sim_id = 'VAN-2|EF05|WZR0058|E58-88'
pred_id = 43225
preds.loc[preds['sim_idx']==sim_id, to_print]

,sim_idx,pred_idx,pred_CoolingTime,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
418,VAN-2|EF05|WZR0058|E58-88,43225,2079.301841,20.0,pwr,bwr,False,4.5,4.4,0.022222,43520.0,36458.87,0.16225,Vandellos-2_PWR,ge7x7-0,8.455988


In [40]:
y_mes = sfco.loc[sfco.index == sim_id].squeeze().drop(lbls)
y_sim = XY.loc[XY.index == pred_id].squeeze().drop(lbls+nonlbls)
y_logpdf = logpdf_df.loc[logpdf_df.index == sim_id].squeeze()
df = pd.DataFrame([y_mes, y_sim])
df.loc['Abs Diff'] = np.abs(df.loc[pred_id] - df.loc[sim_id])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc[sim_id]
df.loc['LogPDF'] = y_logpdf

In [41]:
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.000000,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554
43225,0.902267,0.013397,0.539601,0.015672,103.606336,38.624874,0.123802,0.327600,0.194550,0.097990
Abs Diff,0.119161,0.000530,0.008650,0.015672,5.966218,1.751717,0.001011,0.063279,0.010511,0.007436
% Diff,11.666129,4.117060,1.577808,inf,6.110417,4.750657,0.809630,16.188873,5.711430,8.212038
LogPDF,-1.308784,6.076804,2.642320,NaN,-3.227022,-1.988464,4.152545,-4.269325,3.130042,3.247873
